In [1]:
import zipfile
import pandas as pd
import numpy as np
import nltk as nl
import os
pd.set_option('display.max_columns', None)


In [2]:
dataset_name = 'job_salary'
zf = zipfile.ZipFile('../data/'+dataset_name+ '.zip')
files = zf.infolist()

for f in files:
    print("file present here is", f.filename)
    df = pd.read_csv(zf.open(f.filename))


file present here is Train_rev1.csv


In [3]:
df.shape

(244768, 12)

In [4]:
df_chosen = df.sample(2500, random_state=99)

In [5]:
col_names = ['location','contract_type','contract_time','type_of_job','salary']
num_data = df_chosen[['LocationNormalized','ContractType','ContractTime','Category','SalaryNormalized']]
num_data.columns = col_names

In [6]:
p=np.percentile(num_data['salary'],75)
def target(t):
    if t>p:
        return 1
    else:
        return 0
    
num_data['target'] = num_data['salary'].map(target)

C:\Users\Preetika\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
high_cost = ['London','Milton Keynes','Bath','Reading','Aberdeen','Cambridge','Oxford','Portsmouth','Edinburgh','York']
def location_class(s):
    if s in high_cost:
        return 1
    else:
        return 0
num_data['location_class'] = num_data['location'].map(location_class)

C:\Users\Preetika\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [8]:
num_data.pop('location')
num_data.head()

,contract_type,contract_time,type_of_job,salary,target,location_class
143622,NaN,permanent,"PR, Advertising & Marketing Jobs",33000,0,0
161951,NaN,contract,IT Jobs,54000,1,0
58340,NaN,permanent,Graduate Jobs,17000,0,0
165857,part_time,NaN,Teaching Jobs,23040,0,0
107270,NaN,permanent,IT Jobs,31500,0,0


In [9]:
num_data[['contract_type', 'contract_time','type_of_job']].describe()

,contract_type,contract_time,type_of_job
count,652,1861,2500
unique,2,2,29
top,full_time,permanent,IT Jobs
freq,579,1552,375


### Since, many of the observations in 'contract_time', 'contract_type' are null, let's impute them using the most frequent value for  them

In [10]:
num_data['contract_time'] = num_data['contract_time'].fillna('permanent')
num_data['contract_type'] = num_data['contract_type'].fillna('full_time')
num_data[['contract_type', 'contract_time','type_of_job']].isnull().sum()

C:\Users\Preetika\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Preetika\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


contract_type    0
contract_time    0
type_of_job      0
dtype: int64

In [11]:
df_model = num_data[['contract_type', 'contract_time', 'type_of_job','location_class','target']]
categorical_columns = ['contract_type', 'contract_time', 'type_of_job']
#num_data['contract_type'] = num_data['contract_time'].astype('category')
df_model = pd.get_dummies(df_model, columns= categorical_columns)

In [12]:
df_model.shape   ## Numerical data tranformed to dummy variables

(2500, 35)

### Now we include Text data, and make a boolean vector for the corpus' vocabulary
### Using a Bag o' Words approach to transform text to numerics

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = list(df_chosen['FullDescription'])
vectorizer = CountVectorizer()
corpus_bool = vectorizer.fit_transform(corpus).todense() 
corpus_bool[:10]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
corpus_bool.shape

(2500, 17985)

In [15]:
word_df = pd.DataFrame(vectorizer.fit_transform(corpus).todense(), columns= vectorizer.vocabulary_ )
dict_ = vectorizer.vocabulary_
dict_sorted = sorted(dict_ , key= lambda x: dict_[x], reverse=True) ## Sorting the dictionary based on frequency of words
dict_sorted = dict_sorted[:3000] ## Retaining only 3000 words
dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
dict_filtrered = dictfilt(dict_, dict_sorted)

In [16]:
keys = list(dictfilt(dict_, dict_sorted).keys())
word_df = word_df[keys]

In [17]:
print(word_df.shape)
word_df.head()

(2500, 3000)


strategist  wnac  the  with  their  this  task  to  strategy  who  \
0           0     0    0     0      0     0     0   0         0    0   
1           0     0    0     0      0     0     0   0         0    0   
2           0     0    0     0      0     0     0   0         0    0   
3           0     0    0     0      0     0     0   0         0    0   
4           0     0    0     0      0     0     0   0         0    0   

   stepping  stone  strong  value  success  successful  will  work  well  \
0         0      0       0      0        0           0     0     0     0   
1         0      0       0      0        0           0     0     0     0   
2         0      0       0      0        0           0     0     0     0   
3         0      0       0      0        0           0     0     0     0   
4         0      0       0      0        0           0     0     0     0   

   team  your  working  you  today  step  towards  want  thank  uk  that  \
0     0     0        0    0      0     0        0     0      0   0     0   
1     0     0        0    0      0     0        0     0      0   0     0   
2     0     0        0    0      0     0        0     0      0   0     0   
3     0     0        0    0      0     1        0     0      0   0     0   
4     0     0        0    0      0     1        0     0      0   0     0   

   treated  strictest  without  we  whose  vacancies  visit  website  www  \
0        0          0        0   0      0          0      0        0    0   
1        0          0        0   0      0          0      0        0    0   
2        0          0        0   0      0          0      0        0    0   
3        0          0        0   0      0          0      0        0    0   
4        0          0        0   0      0          0      0        0    0   

   systems  solid  windows  switches  would  volumes  they  weekly  time  \
0        0      0        0         0      0        0     0       0     0   
1        0      0        0         0      0        0     0       0     0   
2        0      0        0         0      0        0     0       0     0   
3        0      0        0         0      0        0     0       0     0   
4        0      0        0         0      0        0     0       0     0   

   starters  telephone  tax  student  timesheets  ssp  word  under  vacancy  \
0         0          0    0        0           0    0     0      0        0   
1         0          0    0        0           0    0     0      0        0   
2         0          0    0        0           0    0     0      0        0   
3         0          0    0        0           0    0     0      0        0   
4         0          0    0        0           0    0     0      0        0   

   was  totaljobs  title  stl  supporting  teaching  staff  tutors  trainers  \
0    0          0      0    0           0         0      0       0         0   
1    0          0      0    0           0         0      0       0         0   
2    0          0      0    0           0         0      0       0         0   
3    0          0      0    0           0         0      0       0         0   
4    0          0      0    0           0         0      0       0         0   

   stating  which  subjects  undertaken  workers  software  world  specialist  \
0        0      0         0           0        0         0      0           0   
1        0      0         0           0        0         0      0           0   
2        0      0         0           0        0         0      0           0   
3        0      0         0           0        0         0      0           0   
4        0      0         0           0        0         0      0           0   

   wide  take  using  sql  visual  studio  xhtml  xml  technologies  \
0     0     0      0    0       0       0      0    0             0   
1     0     0      0    0       0       0      0    0             0   
2     0     0      0    0       0       0      0    2             0   
3     0  

In [18]:
df_model.rename(columns={'target':'my_response_variable'}, inplace=True)
df_model = df_model.reset_index(drop=True)
word_df= word_df.reset_index(drop=True)

In [19]:
full_data = df_model.join(word_df)
full_data.shape

(2500, 3035)

## Running Classification Task

In [20]:
Y_target = full_data['my_response_variable']
full_data.drop('my_response_variable', inplace=True, axis=1)


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(full_data, Y_target, test_size=500, random_state=42)

C:\Users\Preetika\Anaconda2\envs\py36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [22]:
## User Defined Function for model training
def fit_the_model(X_train, y_train, model_name):
    if(model_name == 'NB'):
        clf = BernoulliNB()
        clf.fit(X_train, y_train)
        print(" Model fitting done by Naive Bayes Bernoulli")
        
    elif(model_name == 'RF'):
        clf = RandomForestClassifier()
        clf.fit(X_train, y_train)
        print("Model fitting done by Random Forest")
        
    return clf
## User Defined Function for model evaluation

def classification_model_evaluation(model, X_train, y_train, X_test, y_test):
    prediction_train = model.predict(X_train)
    prediction_test = model.predict(X_test)
    print ("Accuracy on Training data is", metrics.accuracy_score(y_train, prediction_train))
    print ("Accuracy on Test data is", metrics.accuracy_score(y_test, prediction_test))
    print('\n')
    print("Confusion Matrix FOR TEST Obtained is: \n ", confusion_matrix(y_test, prediction_test))
    print('\n')
    print("Confusion Matrix FOR TRAIN Obtained is: \n ", confusion_matrix(y_train, prediction_train))
    report = classification_report(y_test, prediction_test)
    print('\n')
    print("Classification Report \n", report)
    return ("Printed All the metrics for your classification model")

In [23]:
my_mod = fit_the_model(X_train, y_train, 'NB')
classification_model_evaluation(my_mod,X_train, y_train, X_test, y_test )

 Model fitting done by Naive Bayes Bernoulli
Accuracy on Training data is 0.846
Accuracy on Test data is 0.776


Confusion Matrix FOR TEST Obtained is: 
  [[339  41]
 [ 71  49]]


Confusion Matrix FOR TRAIN Obtained is: 
  [[1434   78]
 [ 230  258]]


Classification Report 
              precision    recall  f1-score   support

          0       0.83      0.89      0.86       380
          1       0.54      0.41      0.47       120

avg / total       0.76      0.78      0.76       500



'Printed All the metrics for your classification model'

In [24]:
my_rf_mod = fit_the_model(X_train, y_train, 'RF')
classification_model_evaluation(my_rf_mod,X_train, y_train, X_test, y_test )

Model fitting done by Random Forest
Accuracy on Training data is 0.9775
Accuracy on Test data is 0.768


Confusion Matrix FOR TEST Obtained is: 
  [[367  13]
 [103  17]]


Confusion Matrix FOR TRAIN Obtained is: 
  [[1512    0]
 [  45  443]]


Classification Report 
              precision    recall  f1-score   support

          0       0.78      0.97      0.86       380
          1       0.57      0.14      0.23       120

avg / total       0.73      0.77      0.71       500



'Printed All the metrics for your classification model'

### It is evident that Naive Bayes performs better , as it doesn't result in overfittng, unlike Random Forest